# Atomic and molecular integral with pySCF

In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

In [2]:
import qib

### Construct the molecule

In [3]:
atom = '''H 0 0 0; H 0 0 0.735'''
basis = 'sto-6g'
unit = 'angstrom'
charge = 0
spin = 0
verbose = 0

mol = gto.Mole()
mol.build(atom    = atom,
          basis   = basis,
          charge  = charge,
          spin    = spin,
          unit    = unit,
          verbose = verbose)

### Create the atomic integrals

In [4]:
# costant energy term (nuclear repulsion and kinetic term)
h0 = mol.get_enuc()
# one body integral in the spin-orbital basis (electronic kinetic term and electronic-nuclear attraction)
h1 = np.kron(mol.get_hcore(), np.identity(2))
# two body integral in the spin-orbital basis (electronic repulsion)
h2 = mol.intor('int2e_spinor')

# create a MolecularHamiltonian object starting from the atomic integrals (physicists' convention)
latt = qib.lattice.LayeredLattice(qib.lattice.FullyConnectedLattice((2,)), 2)
field = qib.field.Field(qib.field.ParticleType.FERMION, latt)
symm = [qib.operator.MolecularHamiltonianSymmetry.HERMITIAN, qib.operator.MolecularHamiltonianSymmetry.VARCHANGE]
H = qib.operator.MolecularHamiltonian(field, h0, h1, h2.transpose(0, 2, 3, 1), symm)

In [5]:
print(H.as_matrix())

  (0, 0)	(0.7199689944489797+0j)
  (1, 1)	(-0.4084740771933558+0j)
  (1, 4)	(-0.9679805651651898+0j)
  (2, 2)	(-0.4084740771933558+0j)
  (2, 8)	(-0.9679805651651898+0j)
  (3, 3)	(-2.311915670133334+0j)
  (3, 6)	(1.4152449303153478+0j)
  (3, 9)	(-1.4152449303153478+0j)
  (3, 12)	(-0.30060831485471257+0j)
  (4, 1)	(-0.9679805651651898+0j)
  (4, 4)	(-0.4084740771933558+0j)
  (5, 5)	(-1.8081873720319481+0j)
  (6, 3)	(1.4152449303153476+0j)
  (6, 6)	(-2.1087956868866606+0j)
  (6, 9)	(0.30060831485471257+0j)
  (6, 12)	(1.4152449303153476+0j)
  (7, 7)	(-4.283507503022896+0j)
  (7, 13)	(1.8625092954655056+0j)
  (8, 2)	(-0.9679805651651898+0j)
  (8, 8)	(-0.4084740771933558+0j)
  (9, 3)	(-1.4152449303153476+0j)
  (9, 6)	(0.30060831485471257+0j)
  (9, 9)	(-2.1087956868866606+0j)
  (9, 12)	(-1.4152449303153476+0j)
  (10, 10)	(-1.8081873720319481+0j)
  (11, 11)	(-4.2835075030228955+0j)
  (11, 14)	(1.8625092954655056+0j)
  (12, 3)	(-0.30060831485471257+0j)
  (12, 6)	(1.4152449303153478+0j)
  (12, 9)

### Hartree-Fock calculation

In [6]:
# RHF == restricted HF (look also ROHF, UHF...)
mf = scf.RHF(mol)
# performs the HF calculation
mf.kernel()
# gets the coefficient matrix for the molecular orbitals
coeff = mf.mo_coeff
print(coeff)

[[ 0.5483259   1.21806548]
 [ 0.5483259  -1.21806548]]


### Molecular integrals

In [7]:
spin_coeff = np.kron(coeff, np.identity(2))
h1_mo = np.einsum('ji,jk,kl->il', spin_coeff.conj(), h1, spin_coeff)
h2_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', h2, spin_coeff, spin_coeff, spin_coeff, spin_coeff)
H.tkin = h1_mo
H.vint = h2_mo.transpose(0, 2, 3, 1)
print(h1_mo)
print(h2_mo)

[[-1.26062688e+00  0.00000000e+00 -3.33066907e-16  0.00000000e+00]
 [ 0.00000000e+00 -1.26062688e+00  0.00000000e+00 -3.33066907e-16]
 [-1.11022302e-16  0.00000000e+00 -4.76151148e-01  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00 -4.76151148e-01]]
[[[[ 6.75656092e-01+0.j  0.00000000e+00+0.j -2.77555756e-17+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  6.75656092e-01+0.j  0.00000000e+00+0.j
    -2.77555756e-17+0.j]
   [ 2.77555756e-17+0.j  0.00000000e+00+0.j  6.65257654e-01+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  2.77555756e-17+0.j  0.00000000e+00+0.j
     6.65257654e-01+0.j]]

  [[ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]]

  [[ 2.77555756

### Jordan-Wigner mapping

In [8]:
pauli_op = qib.transform.jordan_wigner_encode_field_operator(H.as_field_operator())

# We must assign a lattice to the pauli operator and the number of sites in the lattice must be exactly like the size of the Pauli strings
field_q = qib.field.Field(qib.field.ParticleType.QUBIT, qib.lattice.LayeredLattice(qib.lattice.FullyConnectedLattice((2,)), 2))
pauli_op.set_field(field_q)
print(pauli_op)

Pauli operator consisting of weighted Pauli strings
    (-1.93541624227124+0j)*IIII
   (1.0865517780771003+0j)*ZIII
   (1.0865517780771003+0j)*IZII
   (0.7004444459506683+0j)*IIZI
   (0.7004444459506688+0j)*IIIZ
 (-0.16891402311629494+0j)*ZZII
 (-0.12100990274925519+0j)*ZIZI
 (-0.16631441337615077+0j)*ZIIZ
(-0.045304510626895576+0j)*XYYX
 (0.045304510626895576+0j)*XXYY
 (0.045304510626895576+0j)*YYXX
(-0.045304510626895576+0j)*YXXY
 (-0.16631441337615077+0j)*IZZI
 (-0.12100990274925519+0j)*IZIZ
 (-0.17504455596821475+0j)*IIZZ

